In [216]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist, squareform, pdist

In [217]:
data = pd.read_csv('data/countries.csv')

In [218]:
data = data[['latitude', 'longitude', 'name']]

In [219]:
data.isna().sum()

latitude     1
longitude    1
name         0
dtype: int64

In [220]:
data = data.dropna()

In [221]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 244 entries, 0 to 244
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   244 non-null    float64
 1   longitude  244 non-null    float64
 2   name       244 non-null    object 
dtypes: float64(2), object(1)
memory usage: 7.6+ KB


# 1. Data Visualization

In [234]:
fig = px.scatter_mapbox(data, 
                        lat='latitude', 
                        lon='longitude', 
                        hover_name='name', 
                        mapbox_style='carto-positron', 
                        zoom=1)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Data Preprocessing

In [223]:
data_train = data.copy()

In [224]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_train[['latitude', 'longitude']])

In [225]:
data_scaled

array([[0.77090111, 0.50132483],
       [0.64575977, 0.64784919],
       [0.71457338, 0.68672547],
       [0.60411665, 0.32352555],
       [0.61170633, 0.3199577 ],
       [0.76178546, 0.55339525],
       [0.75468991, 0.62314263],
       [0.57247665, 0.30315466],
       [0.41915159, 0.5469605 ],
       [0.        , 0.49663307],
       [0.24105872, 0.31842068],
       [0.39907182, 0.01969843],
       [0.80342622, 0.53763888],
       [0.32706203, 0.8720049 ],
       [0.57440742, 0.30060747],
       [0.75517422, 0.63026251],
       [0.77986446, 0.54641415],
       [0.57881028, 0.32984472],
       [0.6474673 , 0.75023711],
       [0.82297836, 0.50936919],
       [0.57255684, 0.4924538 ],
       [0.77212907, 0.56830815],
       [0.66216202, 0.63884663],
       [0.47039113, 0.58074063],
       [0.55337782, 0.50332803],
       [0.70398648, 0.3152216 ],
       [0.52214568, 0.81858639],
       [0.38585768, 0.31849926],
       [0.39930721, 0.35120913],
       [0.65629744, 0.27977585],
       [0.

# 2. Clustering

## 2.1 K-means

In [226]:
model1 = KMeans(n_clusters=4, random_state=10)

In [227]:
data_train['kmeans_group'] = model1.fit_predict(data_scaled)

In [228]:
fig1 = px.scatter_mapbox(data_train, 
                        lat='latitude', 
                        lon='longitude', 
                        hover_name='name', 
                        color='kmeans_group',
                        mapbox_style='carto-positron', 
                        zoom=1)
fig1.update_layout(margin={"r":0,"t":30,"l":0,"b":10}, title='Clustering using KMeans with 4 cluster')
fig1.show()

## 2.2 DBSCAN

In [229]:
dbscan = DBSCAN(eps=0.1, min_samples=3)
data_train['dbscan_group'] = dbscan.fit_predict(data_scaled)

In [230]:
fig2 = px.scatter_mapbox(data_train, 
                        lat='latitude', 
                        lon='longitude', 
                        hover_name='name', 
                        color='dbscan_group',
                        mapbox_style='carto-positron', 
                        zoom=1)
fig2.update_layout(margin={"r":0,"t":30,"l":0,"b":10}, title='Clustering using DBScan')
fig2.show()

## 2.3 Divisive Partitioning Clustering (DPC) 

In [231]:
def dpc_clustering(data, max_clusters):
    def split_cluster(points):
        if len(points) <= 1:
            return [points]
        # Tính toán ma trận khoảng cách giữa các điểm
        distances = squareform(pdist(data[points], 'euclidean'))
        # Tìm hai điểm xa nhất trong cụm
        farthest_points = np.unravel_index(np.argmax(distances), distances.shape)
        # Gán điểm xa nhất vào hai cụm ban đầu
        cluster1 = [points[farthest_points[0]]]
        cluster2 = [points[farthest_points[1]]]
        # Gán các điểm còn lại vào cụm gần nhất
        for i in range(len(points)):
            if i != farthest_points[0] and i != farthest_points[1]:
                if distances[farthest_points[0], i] < distances[farthest_points[1], i]:
                    cluster1.append(points[i])
                else:
                    cluster2.append(points[i])
        return [cluster1, cluster2]

    # Khởi tạo cụm lớn bao gồm tất cả các điểm
    clusters = [list(range(len(data)))]
    while len(clusters) < max_clusters:
        # Tìm cụm lớn nhất và phân chia nó nếu có nhiều hơn một điểm
        largest_cluster = max(clusters, key=len)
        if len(largest_cluster) <= 1:
            break
        clusters.remove(largest_cluster)
        split = split_cluster(largest_cluster)
        clusters.extend(split)

    # Tạo nhãn cụm cho mỗi điểm
    cluster_labels = np.zeros(len(data), dtype=int)
    for cluster_id, cluster in enumerate(clusters):
        for index in cluster:
            cluster_labels[index] = cluster_id

    return cluster_labels

In [232]:
data_train['dpc_group'] = dpc_clustering(data_scaled, max_clusters=4)

In [233]:
fig3 = px.scatter_mapbox(data_train, 
                        lat='latitude', 
                        lon='longitude', 
                        hover_name='name', 
                        color='dpc_group',
                        mapbox_style='carto-positron', 
                        zoom=1)
fig3.update_layout(margin={"r":0,"t":30,"l":0,"b":10}, title='Clustering using DPC with 4 cluster')
fig3.show()

# 3. So sánh và nhận xét
- K-means: Thường tạo ra các cụm có kích thước và hình dạng đồng nhất. Khi áp dụng với dữ liệu địa lý như các tọa độ vĩ độ và kinh độ, các cụm sẽ có xu hướng có hình dạng tròn. Các cụm nằm gần nhau về mặt địa lý sẽ được nhóm lại, nhưng các điểm ngoại lai hoặc cụm không đều sẽ không được xử lý tốt.
- DBSCAN: Phù hợp hơn với dữ liệu địa lý vì có thể phát hiện và loại bỏ các điểm ngoại lai. Các cụm có thể có hình dạng bất kỳ, chẳng hạn như các cụm quốc gia theo khu vực địa lý.
- DPC: DPC có thể bắt đầu với việc phân chia các quốc gia thành các khu vực lớn và sau đó phân chia thêm nếu cần thiết. Điều này có thể tạo ra các cụm mang tính chất phân cấp, nhưng cũng có thể gặp khó khăn trong việc xác định số lượng cụm hoặc tiêu chí dừng phù hợp.